Imports

In [107]:
#Base
import pandas as pd
import numpy as np
import os
import datetime

# Data preparation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.decomposition import TruncatedSVD

Models

In [108]:
#Models
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
# from tensorflow import keras
# from tensorflow.keras import layers

# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.pipeline import Pipeline
# from scikeras.wrappers import KerasClassifier

Read Data

In [109]:
data = pd.read_csv("train.csv")
print(f"The shape of Training data is {data.shape}")

target= data.iloc[:,-1]
data=data.iloc[:,:-1]
# target = data[target_col]
# data = data.drop(target_col, axis=1)
data.head()

The shape of Training data is (233154, 41)


,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1


EDA

In [110]:
data.columns[data.isna().any()].tolist()

['Employment.Type']

In [111]:
cat = """
asset_cost
Current_pincode_ID
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS
disbursed_amount
ltv
NEW.ACCTS.IN.LAST.SIX.MONTHS
NO.OF_INQUIRIES
PERFORM_CNS.SCORE
PRI.ACTIVE.ACCTS
PRI.CURRENT.BALANCE
PRI.DISBURSED.AMOUNT
PRI.NO.OF.ACCTS
PRI.OVERDUE.ACCTS
PRI.SANCTIONED.AMOUNT
PRIMARY.INSTAL.AMT
SEC.ACTIVE.ACCTS
SEC.CURRENT.BALANCE
SEC.DISBURSED.AMOUNT
SEC.INSTAL.AMT
SEC.NO.OF.ACCTS
SEC.OVERDUE.ACCTS
SEC.SANCTIONED.AMOUNT
"""

print("','".join(cat.split("\n")))

','asset_cost','Current_pincode_ID','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','disbursed_amount','ltv','NEW.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES','PERFORM_CNS.SCORE','PRI.ACTIVE.ACCTS','PRI.CURRENT.BALANCE','PRI.DISBURSED.AMOUNT','PRI.NO.OF.ACCTS','PRI.OVERDUE.ACCTS','PRI.SANCTIONED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.ACTIVE.ACCTS','SEC.CURRENT.BALANCE','SEC.DISBURSED.AMOUNT','SEC.INSTAL.AMT','SEC.NO.OF.ACCTS','SEC.OVERDUE.ACCTS','SEC.SANCTIONED.AMOUNT','


Removing columns

In [112]:
remove_cols = ['branch_id','Current_pincode_ID','Employee_code_ID','UniqueID',"supplier_id","manufacturer_id","State_ID"]

data = data.drop(remove_cols, axis=1)

print("Shape of data after removing non-necessary columns: ", data.shape)


Shape of data after removing non-necessary columns:  (233154, 33)


Feature Extraction & Cleaning

In [113]:
def get_age_from_dob(row):
  dob = row['Date.of.Birth']
  disb = row['DisbursalDate']

  dob_year = int(dob.split("-")[2])
  if dob_year < 49:
    dob_year = 2000 + dob_year
  else:
    dob_year = 1900 + dob_year
  
  disb_year = 2000 + int(disb.split("-")[2])

  age_at_disbursement = disb_year - dob_year
  return age_at_disbursement


def get_avg_acc_age_in_months(row):
  avg_age = row['AVERAGE.ACCT.AGE']
  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

def get_credit_age_in_months(row):
  avg_age = row['CREDIT.HISTORY.LENGTH']
  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

def clean_employment_type(row):
  if row['Employment.Type'] not in ['Self employed', 'Salaried']:
    return 'NOT Defined'
  else:
    return row['Employment.Type']

  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

data["age_at_disbursement"] = data.apply (lambda row: get_age_from_dob(row), axis=1)
data["AVERAGE.ACCT.AGE_Months"] = data.apply(lambda row: get_avg_acc_age_in_months(row), axis=1)
data["credit_history_in_months"] = data.apply (lambda row: get_credit_age_in_months(row), axis=1)
data["Employment.Type"] = data.apply (lambda row: clean_employment_type(row), axis=1)

data = data.drop(['DisbursalDate','Date.of.Birth','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH'], axis=1)

print("Shape of data after feature extraction: ", data.shape)
data.head()

Shape of data after feature extraction:  (233154, 32)


,disbursed_amount,asset_cost,ltv,Employment.Type,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,age_at_disbursement,AVERAGE.ACCT.AGE_Months,credit_history_in_months
0,50578,58400,89.55,Salaried,1,1,0,0,0,0,...,0,0,0,0,0,0,0,34,0,0
1,47145,65550,73.23,Self employed,1,1,0,0,0,0,...,0,0,1991,0,0,1,0,33,23,23
2,53278,61360,89.63,Self employed,1,1,0,0,0,0,...,0,0,0,0,0,0,0,33,0,0
3,57513,66113,88.48,Self employed,1,1,0,0,0,0,...,0,0,31,0,0,0,1,25,8,15
4,52378,60300,88.39,Self employed,1,1,0,0,0,0,...,0,0,0,0,0,0,1,41,0,0


In [114]:
#print(data.tail())

One Hot Encoding





In [115]:
categorical_cols = ['Aadhar_flag','Driving_flag','Employment.Type',
                    'MobileNo_Avl_Flag','PAN_flag','Passport_flag',
                    'PERFORM_CNS.SCORE.DESCRIPTION','VoterID_flag'
]
for i in categorical_cols:
    cols = pd.get_dummies(data[i], prefix=i, drop_first=False)
    data = pd.concat([data, cols], axis=1)    

data = data.drop(categorical_cols, axis=1)
print("Shape of data after one-hot encoding: ", data.shape)
data.head()

Shape of data after one-hot encoding:  (233154, 58)


,disbursed_amount,asset_cost,ltv,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,...,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available,VoterID_flag_0,VoterID_flag_1
0,50578,58400,89.55,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,47145,65550,73.23,598,1,1,1,27600,50200,50200,...,0,0,0,0,0,0,0,0,1,0
2,53278,61360,89.63,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,57513,66113,88.48,305,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,52378,60300,88.39,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


Scaling

In [99]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_cols = ['asset_cost','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','disbursed_amount','ltv',
              'NEW.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES','PERFORM_CNS.SCORE','PRI.ACTIVE.ACCTS','PRI.CURRENT.BALANCE',
              'PRI.DISBURSED.AMOUNT','PRI.NO.OF.ACCTS','PRI.OVERDUE.ACCTS','PRI.SANCTIONED.AMOUNT','PRIMARY.INSTAL.AMT',
              'SEC.ACTIVE.ACCTS','SEC.CURRENT.BALANCE',
              'SEC.DISBURSED.AMOUNT','SEC.INSTAL.AMT','SEC.NO.OF.ACCTS','SEC.OVERDUE.ACCTS','SEC.SANCTIONED.AMOUNT']
# scaler = StandardScaler()
col_list = data.columns.values.tolist()
print(col_list)
data[scale_cols] = scaler.fit_transform(data[scale_cols])
data_n = scaler.fit_transform(data)
data_n = pd.DataFrame(data_n, columns = col_list)
data = data_n

['disbursed_amount', 'asset_cost', 'ltv', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES', 'age_at_disbursement', 'AVERAGE.ACCT.AGE_Months', 'credit_history_in_months', 'Aadhar_flag_0', 'Aadhar_flag_1', 'Driving_flag_0', 'Driving_flag_1', 'Employment.Type_NOT Defined', 'Employment.Type_Salaried', 'Employment.Type_Self employed', 'MobileNo_Avl_Flag_1', 'PAN_flag_0', 'PAN_flag_1', 'Passport_flag_0', 'Passport_flag_1', 'PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk', 'PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk', 'PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk', 'PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk', 'PERFORM_CNS.SCORE

In [100]:
# data_n.head()


Splitting Data

In [116]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify=target)

print("Shape of data:", data.shape)
print("Shape of target", target.shape)
print("Shape of Xtrain and ytrain:", X_train.shape, y_train.shape)
print("Shape of Xtest and ytest:", X_test.shape, y_test.shape)

Shape of data: (233154, 58)
Shape of target (233154,)
Shape of Xtrain and ytrain: (186523, 58) (186523,)
Shape of Xtest and ytest: (46631, 58) (46631,)


In [117]:
X_train.head()

,disbursed_amount,asset_cost,ltv,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,...,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available,VoterID_flag_0,VoterID_flag_1
192380,57159,67730,88.59,680,5,5,0,63885,101679,101679,...,0,0,0,0,0,0,0,0,1,0
43291,46645,76706,65.18,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
210644,30484,74642,41.53,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
50259,62447,92130,70.01,749,3,2,0,40404,61100,61100,...,0,0,0,0,0,0,0,0,1,0
158140,54515,64500,87.60,300,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0


XGBoost

In [118]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_y_pred = xgb.predict(X_test)

cnf_matrix = metrics.confusion_matrix(y_test, xgb_y_pred)
print(cnf_matrix)


print("Accuracy:",metrics.accuracy_score(y_test, xgb_y_pred))
print("Precision:",metrics.precision_score(y_test, xgb_y_pred))
print("Recall:",metrics.recall_score(y_test, xgb_y_pred))

C:\Users\Checkout\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:02:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[36258   251]
 [ 9924   198]]
Accuracy: 0.7817975166734575
Precision: 0.44097995545657015
Recall: 0.01956135151155898


In [119]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=1200,max_depth=7)
model.fit(X_train,y_train)
ypred=model.predict(X_test)


In [120]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, ypred)
print("Accuracy:",metrics.accuracy_score(y_test, ypred))

Accuracy: 0.7829340996332912


In [121]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=51)
neigh.fit(X_train,y_train)
ypred=neigh.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, ypred)
print("Accuracy:",metrics.accuracy_score(y_test, ypred))

Accuracy: 0.7828054298642534


In [122]:
print(X_train.shape,X_test.shape)

(186523, 58) (46631, 58)


In [123]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [124]:
ann=Sequential()
nodes=20
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))
ann.add(Dense(units=nodes, activation='relu'))

ann.add(Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 500, epochs = 1000)

Epoch 1/1000
374/374 [==============================] - 2s 3ms/step - loss: 2.4463 - accuracy: 0.7559
Epoch 2/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5522 - accuracy: 0.7800: 0s - loss: 0.5515 - accura
Epoch 3/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.6049 - accuracy: 0.7736: 0s - loss: 0.5369  - ETA: 0s - loss: 0.6120 - accuracy: 
Epoch 4/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5419 - accuracy: 0.7809
Epoch 5/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5251 - accuracy: 0.7818
Epoch 6/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5280 - accuracy: 0.7818
Epoch 7/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5386 - accuracy: 0.7801
Epoch 8/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5278 - accuracy: 0.7821
Epoch 9/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5281 - accuracy: 0.7

374/374 [==============================] - 1s 3ms/step - loss: 0.5131 - accuracy: 0.7829
Epoch 74/1000
374/374 [==============================] - ETA: 0s - loss: 0.5126 - accuracy: 0.7828 ETA: 0s - loss: 0.5134 - accuracy:  - ETA: 0s - loss: 0.5126  - 1s 3ms/step - loss: 0.5125 - accuracy: 0.7829
Epoch 75/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5122 - accuracy: 0.7829: 1s - loss: - ETA: 0s - loss: 0.5106 - 
Epoch 76/1000
374/374 [==============================] - ETA: 0s - loss: 0.5118 - accuracy: 0.7830 ETA: 0s - loss: 0.5111 - accura - 1s 3ms/step - loss: 0.5120 - accuracy: 0.7829
Epoch 77/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5124 - accuracy: 0.7829: 0s - loss: 0.5123 - ac
Epoch 78/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5127 - accuracy: 0.7829: 0s - loss: 0.5128 
Epoch 79/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5128 - accuracy: 0.7829: 0s - loss: 0.5 - ETA: 0s -

374/374 [==============================] - 1s 3ms/step - loss: 0.5138 - accuracy: 0.7829
Epoch 142/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5135 - accuracy: 0.7829: 0s - loss: 0.5157 
Epoch 143/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5133 - accuracy: 0.7829
Epoch 144/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5134 - accuracy: 0.7829
Epoch 145/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5139 - accuracy: 0.7829
Epoch 146/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5148 - accuracy: 0.7829
Epoch 147/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5143 - accuracy: 0.7829
Epoch 148/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5134 - accuracy: 0.7829: 0s - loss: 0.5133 - ac
Epoch 149/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5133 - accuracy: 0.7829
Epoch 150/1000
374/374 [===========

374/374 [==============================] - 1s 3ms/step - loss: 0.5118 - accuracy: 0.7829
Epoch 216/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5119 - accuracy: 0.7829
Epoch 217/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5122 - accuracy: 0.7829
Epoch 218/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5147 - accuracy: 0.7828
Epoch 219/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5118 - accuracy: 0.7829: 0s - loss: 0.5123 - 
Epoch 220/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5115 - accuracy: 0.7829: 0s - loss: 0.5134 - 
Epoch 221/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5116 - accuracy: 0.7829
Epoch 222/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5117 - accuracy: 0.7829
Epoch 223/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5116 - accuracy: 0.7829
Epoch 224/1000
374/374 [===========

374/374 [==============================] - 1s 3ms/step - loss: 0.5140 - accuracy: 0.7829: 
Epoch 287/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5135 - accuracy: 0.7829
Epoch 288/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5129 - accuracy: 0.7829: 0s - loss: 0.5123 - accuracy: 0.
Epoch 289/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5123 - accuracy: 0.7829: 0s - loss: 0.5124 - accuracy: 0.
Epoch 290/1000
374/374 [==============================] - ETA: 0s - loss: 0.5120 - accuracy: 0.7828 ETA: 0s - loss: 0.5141 - accuracy:  - ETA: 0s - loss: 0.5114 - accu - ETA: 0s - loss: 0.5118 -  - 1s 3ms/step - loss: 0.5117 - accuracy: 0.7829
Epoch 291/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5117 - accuracy: 0.7829
Epoch 292/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5114 - accuracy: 0.7829
Epoch 293/1000
374/374 [==============================] - 1s 3ms/step - loss:

...

374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 705/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 0s - loss: 0.5247 - 
Epoch 706/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 707/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 708/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 709/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 710/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 711/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 712/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 713/1000
374/374 [==============================] -

374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 771/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 772/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 773/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 774/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 775/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 776/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 777/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 778/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 779/1000
374/374 [==============================] - 1s 3ms/step - loss: 0

374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 841/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 842/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 843/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 844/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 845/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 1s - loss: 0.5276 - accu - ETA: 0s - loss: 0.5246  - ETA: 0s - loss: 0.5220 - accura
Epoch 846/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 847/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 0s - loss: 0.5232 - ac
Epoch 848/1000
374/374 [==============================] - 1s 3ms/step - l

374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 0s - loss: 0.5233 - ac
Epoch 912/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 913/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 914/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 915/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 916/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 0s - loss: 0 - ETA: 0s - loss: 0.5229 - accuracy: 0. - ETA: 0s - loss: 0.5235 - accu
Epoch 917/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 918/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829: 0s - loss: 0.5228 - accuracy: 0.
Epoch 919/1000
374/374 [===============

374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 981/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 982/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 983/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 984/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 985/1000
374/374 [==============================] - 1s 4ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 986/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 987/1000
374/374 [==============================] - 1s 3ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 988/1000
374/374 [==============================] - 1s 4ms/step - loss: 0.5232 - accuracy: 0.7829
Epoch 989/1000
374/374 [==============================] - 1s 4ms/step - loss: 0

In [125]:
ypred=ann.predict(X_test)

In [126]:
print(ypred>0.5)
ypred = [1 if i[0]==True else 0 for i in ypred>0.5]

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [129]:
cnf_matrix = metrics.confusion_matrix(y_test, ypred)
print("accuracy",metrics.accuracy_score(y_test, ypred))

accuracy 0.7829340996332912
